# TFX Keras Component Tutorial

***A Component-by-Component Introduction to TensorFlow Extended (TFX)***

# Upgrade pip

In [1]:
import sys
if 'google.colab' in sys.modules:
  !pip install --upgrade pip

# Install TFX

In [3]:
!pip install -Uq tfx

You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.


## Restart Runtime

# Import Packages

In [4]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

## Check Versions

In [5]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.8.0
TFX version: 1.7.1


# Set Pipeline Paths

In [6]:
# This is the root directory for your TFX pip package installation.
_tfx_root = tfx.__path__[0]

# This is the directory containing the TFX Vitals Pipeline example.
_vitals_root = os.path.join(_tfx_root, 'examples/vitals')

# This is the path where your model will be pushed for serving.
_serving_model_dir = os.path.join(
    tempfile.mkdtemp(), 'serving_model/vitals_simple')

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

In [9]:
# Fetch the Training Data

In [7]:
_attr_root = tempfile.mkdtemp(prefix='tfx-data')
ATTR_PATH = 'https://raw.githubusercontent.com/redhat-na-ssa/mlops-prototype/main/data/dataSepsis/csv_format/attribute_definitions.csv'
_attr_filepath = os.path.join(_attr_root, "data.csv")
urllib.request.urlretrieve(ATTR_PATH, _attr_filepath)

('/tmp/tfx-data5_x44ybo/data.csv', <http.client.HTTPMessage at 0x7f02b8036f70>)

# View the Data

In [12]:
!head {_attr_filepath}

target_file,attribute_name,attribute_definition
,List all the attributes in the dataset. Label continuous attributes with c and discrete with d,
,,
pat_vitals_labeled.csv,Vital signs (columns 1-8),"Doctors order, basis every 4 hours, least invasive, cost least amount of money"
pat_vitals_labeled.csv,HR,Heart rate (beats per minute)
pat_vitals_labeled.csv,O2Sat,Pulse oximetry (%)
pat_vitals_labeled.csv,Temp,Temperature (Deg C)
pat_vitals_labeled.csv,SBP,Systolic BP (mm Hg)
pat_vitals_labeled.csv,MAP,Mean arterial pressure (mm Hg)
pat_vitals_labeled.csv,DBP,Diastolic BP (mm Hg)


In [8]:
_data_root = tempfile.mkdtemp(prefix='tfx-data')
DATA_PATH = 'https://raw.githubusercontent.com/redhat-na-ssa/mlops-prototype/main/data/dataSepsis/csv_format/pat_vitals_labeled-dataSepsis.csv'
_data_filepath = os.path.join(_data_root, "data.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

('/tmp/tfx-datatw8wocny/data.csv', <http.client.HTTPMessage at 0x7f01f299ebe0>)

# View the Data

In [11]:
!head {_data_filepath}

patient_id,record_date,record_time,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,isSepsis
1,,,63,90,40.3,NaN,NaN,NaN,17,NaN,0
2,,,79,95,39.2,143,77,47,13,NaN,0
3,,,87,94,40.3,133,74,48,20,NaN,0
4,,,71,100,42.1,NaN,NaN,NaN,15,NaN,0
5,,,68,94.5,39.7,147.5,102,NaN,20,NaN,0
6,,,78,99,39.6,100,67,49.5,18,NaN,0
7,,,242,NaN,39.30,NaN,NaN,NaN,33,NaN,1
8,,,81,100,40.3,112,79.5,63,18,NaN,0
9,,,178,100,39.22,141,85,57,22,NaN,1


# Create the InteractiveContext

This allow us to run TFX components interactively in this notebook.

In [13]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
context = InteractiveContext()

# ExampleGen

The ExampleGen component is usually at the start of a TFX pipeline. It will:

1. Split data into training and evaluation sets (by default, 2/3 training + 1/3 eval)
1. Convert data into the tf.Example format 
1. Copy data into the _tfx_root directory for other components to access
ExampleGen takes as input the path to your data source. In our case, this is the _data_root path that contains the downloaded CSV.

Note: In this notebook, we can instantiate components one-by-one and run them with InteractiveContext.run(). By contrast, in a production setting, we would specify all the components upfront in a Pipeline to pass to the orchestrator (see the Building a TFX Pipeline Guide). https://www.tensorflow.org/tfx/guide/build_tfx_pipeline

## Enabling the Cache

When using the InteractiveContext in a notebook to develop a pipeline you can control when individual components will cache their outputs. Set enable_cache to True when you want to reuse the previous output artifacts that the component generated. Set enable_cache to False when you want to recompute the output artifacts for a component, if you are making changes to the code for example.

In [14]:
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)
context.run(example_gen, enable_cache=True)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data /tmp/tfx-datatw8wocny/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

Let's examine the output artifacts of `ExampleGen`. This component produces two artifacts, training examples and evaluation examples:

In [16]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /tmp/tfx-interactive-2022-04-21T23_36_39.820592-9u_hwiqo/CsvExampleGen/examples/1


We can also take a look at the first training examples:

In [17]:
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "DBP"
    value {
      float_list {
        value: nan
      }
    }
  }
  feature {
    key: "EtCO2"
    value {
      float_list {
        value: nan
      }
    }
  }
  feature {
    key: "HR"
    value {
      int64_list {
        value: 63
      }
    }
  }
  feature {
    key: "MAP"
    value {
      float_list {
        value: nan
      }
    }
  }
  feature {
    key: "O2Sat"
    value {
      float_list {
        value: 90.0
      }
    }
  }
  feature {
    key: "Resp"
    value {
      int64_list {
        value: 17
      }
    }
  }
  feature {
    key: "SBP"
    value {
      float_list {
        value: nan
      }
    }
  }
  feature {
    key: "Temp"
    value {
      float_list {
        value: 40.29999923706055
      }
    }
  }
  feature {
    key: "isSepsis"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "patient_id"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
 

# StatisticsGen

The StatisticsGen component computes statistics over your dataset for data analysis, as well as for use in downstream components. It uses the TensorFlow Data Validation library.

StatisticsGen takes as input the dataset we just ingested using ExampleGen.

In [18]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen, enable_cache=True)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2022-04-21T23_36_39.820592-9u_hwiqo/StatisticsGen/statistics/2/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /tmp/tfx-interactive-2022-04-21T23_36_39.820592-9u_hwiqo/StatisticsGen/statistics/2/Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

After `StatisticsGen` finishes running, we can visualize the outputted statistics. Try playing with the different plots!

In [19]:
context.show(statistics_gen.outputs['statistics'])

# SchemaGen

The `SchemaGen` component generates a schema based on your data statistics. (A schema defines the expected bounds, types, and properties of the features in your dataset.) It also uses the [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started) library.

Note: The generated schema is best-effort and only tries to infer basic properties of the data. It is expected that you review and modify it as needed.

`SchemaGen` will take as input the statistics that we generated with `StatisticsGen`, looking at the training split by default.

In [21]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen, enable_cache=True)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /tmp/tfx-interactive-2022-04-21T23_36_39.820592-9u_hwiqo/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

After `SchemaGen` finishes running, we can visualize the generated schema as a table.

In [22]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'DBP',FLOAT,required,single,-
'EtCO2',FLOAT,required,single,-
'HR',INT,required,single,-
'MAP',FLOAT,required,single,-
'O2Sat',FLOAT,required,single,-
'Resp',INT,required,single,-
'SBP',FLOAT,required,single,-
'Temp',FLOAT,required,single,-
'isSepsis',INT,required,single,-


Each feature in your dataset shows up as a row in the schema table, alongside its properties. The schema also captures all the values that a categorical feature takes on, denoted as its domain.

# ExampleValidator
The `ExampleValidator` component detects anomalies in your data, based on the expectations defined by the schema. It also uses the [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started) library.

`ExampleValidator` will take as input the statistics from `StatisticsGen`, and the schema from `SchemaGen`.

In [23]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator, enable_cache=True)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to /tmp/tfx-interactive-2022-04-21T23_36_39.820592-9u_hwiqo/ExampleValidator/anomalies/4/Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to /tmp/tfx-interactive-2022-04-21T23_36_39.820592-9u_hwiqo/ExampleValidator/anomalies/4/Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

After `ExampleValidator` finishes running, we can visualize the anomalies as a table.

In [25]:
context.show(example_validator.outputs['anomalies'])

In the anomalies table, we can see that there are no anomalies. This is what we'd expect, since this the first dataset that we've analyzed and the schema is tailored to it. You should review this schema -- anything unexpected means an anomaly in the data. Once reviewed, the schema can be used to guard future data, and anomalies produced here can be used to debug model performance, understand how your data evolves over time, and identify data errors.

# Transform

The `Transform` component performs feature engineering for both training and serving. It uses the [TensorFlow Transform](https://www.tensorflow.org/tfx/transform/get_started) library.

`Transform` will take as input the data from `ExampleGen`, the schema from `SchemaGen`, as well as a module that contains user-defined Transform code.

Let's see an example of user-defined Transform code below (for an introduction to the TensorFlow Transform APIs, [see the tutorial](https://www.tensorflow.org/tfx/tutorials/transform/simple)). First, we define a few constants for feature engineering:

Note: The `%%writefile` cell magic will save the contents of the cell as a `.py` file on disk. This allows the `Transform` component to load your code as a module.

In [26]:
_vitals_constants_module_file = 'vitals_constants.py'

In [27]:
%%writefile {_vitals_constants_module_file}

NUMERICAL_FEATURES = ['HR', 'Resp', 'Temp', 'isSepsis']

# Keys
LABEL_KEY = 'isSepsis'

def t_name(key):
  """
  Rename the feature keys so that they don't clash with the raw keys when
  running the Evaluator component.
  Args:
    key: The original feature key
  Returns:
    key with '_xf' appended
  """
  return key + '_xf'

Writing vitals_constants.py


Next, we write a `preprocessing_fn` that takes in raw data as input, and returns transformed features that our model can train on:

In [28]:
_vitals_transform_module_file = 'vitals_transform.py'

In [36]:
%%writefile {_vitals_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

# Imported files such as vitals_constants are normally cached, so changes are
# not honored after the first import.  Normally this is good for efficiency, but
# during development when we may be iterating code it can be a problem. To
# avoid this problem during development, reload the file.
import vitals_constants
import sys
if 'google.colab' in sys.modules:  # Testing to see if we're doing development
  import importlib
  importlib.reload(vitals_constants)

_NUMERICAL_FEATURES = vitals_constants.NUMERICAL_FEATURES
_LABEL_KEY = vitals_constants.LABEL_KEY


def _fill_in_missing(x):
  """Replace missing values in a SparseTensor.
  Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
  Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.
  Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
  """
  if not isinstance(x, tf.sparse.SparseTensor):
    return x

  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)


def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.
  Args:
    inputs: map from feature keys to raw not-yet-transformed features.
  Returns:
    Map from string feature key to transformed feature operations.
  """
  outputs = {}
  for key in _NUMERICAL_FEATURES:
    # If sparse make it dense, setting nan's to 0 or '', and apply zscore.
    outputs[vitals_constants.t_name(key)] = tft.scale_to_z_score(
        _fill_in_missing(inputs[key]), name=key)

  # Was this passenger a big tipper?
  septic = _fill_in_missing(inputs[_LABEL_KEY])
  #outputs[_LABEL_KEY] = tf.where(
  #    tf.math.is_nan(taxi_fare),
  #    tf.cast(tf.zeros_like(taxi_fare), tf.int64),
  #    # Test if the tip was > 20% of the fare.
  #    tf.cast(
  #        tf.greater(tips, tf.multiply(taxi_fare, tf.constant(0.2))), tf.int64))

  return outputs

Overwriting vitals_transform.py


Now, we pass in this feature engineering code to the `Transform` component and run it to transform your data.

In [37]:
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_vitals_transform_module_file))
context.run(transform, enable_cache=True)

INFO:absl:Generating ephemeral wheel package for '/opt/app-root/src/tfx_keras_component_on_ocp/vitals_septic/vitals_transform.py' (including modules: ['vitals_constants', 'vitals_transform']).
INFO:absl:User module package has hash fingerprint version c2f5ed0d8bac6c93781933a9bc6725bb30e7064aa08193ddddd5c172fa08c3bc.
INFO:absl:Executing: ['/opt/app-root/bin/python3.8', '/tmp/tmpa8ko0oc0/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpauk93krd', '--dist-dir', '/tmp/tmp7hi8zs9h']
INFO:absl:Successfully built user code wheel distribution at '/tmp/tfx-interactive-2022-04-21T23_36_39.820592-9u_hwiqo/_wheels/tfx_user_code_Transform-0.0+c2f5ed0d8bac6c93781933a9bc6725bb30e7064aa08193ddddd5c172fa08c3bc-py3-none-any.whl'; target user module is 'vitals_transform'.
INFO:absl:Full user module path is 'vitals_transform@/tmp/tfx-interactive-2022-04-21T23_36_39.820592-9u_hwiqo/_wheels/tfx_user_code_Transform-0.0+c2f5ed0d8bac6c93781933a9bc6725bb30e7064aa08193ddddd5c172fa08c3bc-py3-none-

Instructions for updating:
Use ref() instead.


INFO:absl:Feature DBP has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature EtCO2 has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature HR has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature MAP has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature O2Sat has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Resp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature SBP has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Temp has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature isSepsis has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature patient_id has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature record_date has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature record_time has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature DBP has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature EtCO2 has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature HR has no

INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2022-04-21T23_36_39.820592-9u_hwiqo/Transform/transform_graph/8/.temp_path/tftransform_tmp/55105e712834428f9bb6c8af9257d991/assets
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:tensorflow_text is not available.
INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2022-04-21T23_36_39.820592-9u_hwiqo/Transform/transform_graph/8/.temp_path/tftransform_tmp/4ffe90d6e8844fadb3362c8c1953dd25/assets
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:tensorflow_text is not available.
INFO:tensorflow:struct2tensor is not available.
INFO:tensorflow:tensorflow_decision_forests is not available.
INFO:tensorflow:tensorflow_text is not available.


INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 8
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

Let's examine the output artifacts of `Transform`. This component produces two types of outputs:

* `transform_graph` is the graph that can perform the preprocessing operations (this graph will be included in the serving and evaluation models).
* `transformed_examples` represents the preprocessed training and evaluation data.

In [38]:
transform.outputs

{'transform_graph': OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}),
 'transformed_examples': OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}),
 'updated_analyzer_cache': OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}),
 'pre_transform_schema': OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}),
 'pre_transform_stats': OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}),
 'post_transform_schema': OutputChannel(arti

Take a peek at the `transform_graph` artifact.  It points to a directory containing three subdirectories.

In [39]:
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

['transformed_metadata', 'transform_fn', 'metadata']

The `transformed_metadata` subdirectory contains the schema of the preprocessed data. The `transform_fn` subdirectory contains the actual preprocessing graph. The `metadata` subdirectory contains the schema of the original data.

We can also take a look at the first three transformed examples:

In [40]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "HR_xf"
    value {
      float_list {
        value: -0.6613534092903137
      }
    }
  }
  feature {
    key: "Resp_xf"
    value {
      float_list {
        value: -0.1939026415348053
      }
    }
  }
  feature {
    key: "Temp_xf"
    value {
      float_list {
        value: 2.2171924114227295
      }
    }
  }
  feature {
    key: "isSepsis_xf"
    value {
      float_list {
        value: -0.27875474095344543
      }
    }
  }
}

features {
  feature {
    key: "HR_xf"
    value {
      float_list {
        value: -0.060083888471126556
      }
    }
  }
  feature {
    key: "Resp_xf"
    value {
      float_list {
        value: -0.9738271832466125
      }
    }
  }
  feature {
    key: "Temp_xf"
    value {
      float_list {
        value: 1.3707969188690186
      }
    }
  }
  feature {
    key: "isSepsis_xf"
    value {
      float_list {
        value: -0.27875474095344543
      }
    }
  }
}

features {
  feature {
    key: "HR_xf"
    va

After the `Transform` component has transformed your data into features, and the next step is to train a model.

# Trainer

The `Trainer` component will train a model that you define in TensorFlow. Default Trainer support Estimator API, to use Keras API, you need to specify [Generic Trainer](https://github.com/tensorflow/community/blob/master/rfcs/20200117-tfx-generic-trainer.md) by setup `custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor)` in Trainer's contructor.

`Trainer` takes as input the schema from `SchemaGen`, the transformed data and graph from `Transform`, training parameters, as well as a module that contains user-defined model code.